In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('CRM win loss 28032017 - Summary.csv',encoding="cp1252")
#df = pd.read_csv('C:\Python\Data\CNG_Recommender_transation01.csv')

In [3]:
df.head()

,Sales Organisation,Owner,SAP Number Exist,Product Exists,Cg InYear or NextYear,Cg Opportunity or Threat,date_diff,Estimated Value,Budget Amount,Opportunity State
0,City and Guilds,Adam Warsop,N,N,Next Year,Opportunity,96,"150,000",NaN,Lost
1,City and Guilds,Adam Warsop,Y,N,In Year,Opportunity,15,"500,000",NaN,Won
2,City and Guilds,Adam Warsop,Y,Y,Both,Opportunity,21,850,NaN,Lost
3,City and Guilds,Adam Warsop,Y,N,Both,Opportunity,-32,"1,100,000",NaN,Won
4,City and Guilds,Amanda Kilpatrick,Y,N,NaN,Opportunity,0,"3,450",NaN,Lost


In [4]:
df.columns

Index(['Sales Organisation', 'Owner', 'SAP Number Exist', 'Product Exists',
       'Cg InYear or NextYear', 'Cg Opportunity or Threat', 'date_diff',
       'Estimated Value', 'Budget Amount', 'Opportunity State'],
      dtype='object')

In [5]:
df = df.replace(np.nan, '', regex=True)
df.head()

,Sales Organisation,Owner,SAP Number Exist,Product Exists,Cg InYear or NextYear,Cg Opportunity or Threat,date_diff,Estimated Value,Budget Amount,Opportunity State
0,City and Guilds,Adam Warsop,N,N,Next Year,Opportunity,96,"150,000",,Lost
1,City and Guilds,Adam Warsop,Y,N,In Year,Opportunity,15,"500,000",,Won
2,City and Guilds,Adam Warsop,Y,Y,Both,Opportunity,21,850,,Lost
3,City and Guilds,Adam Warsop,Y,N,Both,Opportunity,-32,"1,100,000",,Won
4,City and Guilds,Amanda Kilpatrick,Y,N,,Opportunity,0,"3,450",,Lost


In [6]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
d = defaultdict(LabelEncoder)
# Encoding the variable
df_encode1 = df.apply(lambda x: d[x.name].fit_transform(x))

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_encode2 = sc.fit_transform(df_encode1[["date_diff","Estimated Value","Budget Amount"]])
df_encode1[["date_diff","Estimated Value","Budget Amount"]] = pd.DataFrame(df_encode2)
df_encode1.head()

,Sales Organisation,Owner,SAP Number Exist,Product Exists,Cg InYear or NextYear,Cg Opportunity or Threat,date_diff,Estimated Value,Budget Amount,Opportunity State
0,0,0,0,0,3,1,1.826947,0.229015,-1.050666,0
1,0,0,1,0,2,1,-0.231361,1.983809,-1.050666,1
2,0,0,1,1,1,1,0.040644,2.660825,-1.050666,0
3,0,0,1,0,1,1,-0.962121,-0.527764,-1.050666,1
4,0,2,1,0,0,1,-0.519605,1.141430,-1.050666,0


In [9]:
X = df_encode1.iloc[:, 0:9].values
y = df_encode1.iloc[:, 9].values

onehotencoder = OneHotEncoder(categorical_features = [0,1,2,3,4,5])
X = onehotencoder.fit_transform(X).toarray()

# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [12]:
# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 379,  532],
       [ 219, 1924]])

In [13]:
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.708583443181942, 0.65691657877766074, 0.6695115240282592, None)

In [14]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 329,  582],
       [ 154, 1989]])

In [16]:
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.72739417922310723, 0.64463986337976298, 0.65794614046866728, None)

In [17]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [18]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 596,  315],
       [ 313, 1830]])

In [19]:
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.75440620985175433, 0.75408459779959047, 0.75424491553222905, None)

In [20]:
from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

prediction, bias, contributions = ti.predict(rf, instances)

ImportError: No module named 'treeinterpreter'

In [21]:
# Fitting K-NN to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [22]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 501,  410],
       [ 305, 1838]])

In [23]:
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.71960187384650709, 0.7038106350904817, 0.71037146851046917, None)

In [24]:
# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [25]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 575,  336],
       [ 314, 1829]])

In [26]:
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.74579892294063699, 0.74232548419201616, 0.74400340451872482, None)